In [ ]:
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/stroke_research/geneva_stroke_incidence/data/PPGSS_3_ICD10vsICD11.xlsx'

In [ ]:
df = pd.read_excel(data_path)

In [ ]:
df.head()

In [ ]:
outcome = '3M StrokeICH'

In [ ]:
df.dropna(subset=[outcome], inplace=True)

In [ ]:
# binarize outcome
if outcome == '3M StrokeICH':
    df[outcome] = df[outcome].apply(lambda x: 1 if x == 'yes' else 0)

In [ ]:
X = df[['ICD10']]
X = sm.add_constant(X)
y = df[outcome].astype(int)

In [ ]:
model = sm.Logit(y, X)

In [ ]:
result = model.fit()

In [ ]:
result.summary()

In [ ]:
# get all coefficients as a dataframe, along with std, z and confidence intervals 
coefficients = result.params.to_frame().reset_index()

coefficients['std'] = result.bse.to_frame().reset_index()[0]
coefficients['z'] = result.tvalues.to_frame().reset_index()[0]

coefficients['CI_low'] = result.conf_int().reset_index()[0]
coefficients['CI_high'] = result.conf_int().reset_index()[1]

coefficients.columns = ['Predictor', 'Coefficient', 'std', 'z', 'CI_low', 'CI_high']

In [ ]:
coefficients

In [ ]:
# coefficients.to_excel(f'/Users/jk1/Downloads/ICD10_vs_{outcome}.xlsx')

In [ ]:
result.pvalues

In [ ]:
fig = plt.figure(figsize=(10, 6))

# create a barchart of the outcome variable by the ICD10 code
ax = sns.barplot(x='ICD10', y=outcome, data=df, ci=None, hue='ICD10', legend=False)

# add bar with pvalue
x1, x2 = 0, 1
y, h, col = 0.18 + 0.01, 0.01, 'k'
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, f'p={result.pvalues[-1]:.4f}', ha='center', va='bottom', color=col)

In [ ]:
df.groupby('ICD10')[outcome].value_counts(normalize=True)

In [ ]:
# fig.savefig(f'/Users/jk1/Downloads/{outcome}_vs_ICD10.png')

In [ ]:
# make a boxplot with NIHonadmission for with ICD10 vs all patients

fig = plt.figure(figsize=(10, 10))

ICD10_df = df[df['ICD10'] == 1]
ICD10_df['cat'] = 'ICD10'
ICD11_df = df[df['ICD11'] == 1]
ICD11_df['cat'] = 'ICD11'

temp_df = pd.concat([ICD10_df, ICD11_df])
temp_df.reset_index(inplace=True)

ax = sns.barplot(x='cat', y=outcome, data=temp_df, ci=None, hue='cat', legend=False)

In [ ]:
# get median and IQR for both categories
temp_df.groupby('cat')[outcome].value_counts(normalize=True)

### Test whole group vs subgroup
instead of subgroup ICD10 vs subgroup ICD11-ICD10  

In [ ]:
df.head()

In [ ]:
outcome

In [ ]:
# remove space in outcome 
df.rename(columns={outcome: outcome.replace(' ', '')}, inplace=True)
outcome = outcome.replace(' ', '')
if outcome == '3MStrokeICH':
    df.rename(columns={outcome: 'recurrence_90d'}, inplace=True)
    outcome = 'recurrence_90d'

In [ ]:
df.groupby('ICD10vsICD11')[outcome].value_counts(normalize=True)

In [ ]:
import os
os.environ["R_HOME"] = "/Library/Frameworks/R.framework/Resources"
from pymer4.models import Lmer

model = Lmer(f"{outcome}  ~ ICD10  + (1|ID)",
             data=df, family = 'binomial')


In [ ]:
model.fit()

In [ ]:
model.coefs['P-val'][1]

In [ ]:
# model.coefs.to_excel(f'/Users/jk1/Downloads/mixed_effects_ICD10_vs_{outcome}.xlsx')